In [69]:
from faker import Faker
import random
import pandas as pd
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

# Initialize faker
fake = Faker()

class AESCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_EAX)
        ciphertext, tag = cipher.encrypt_and_digest(data)
        return cipher.nonce, b64encode(ciphertext).decode('utf-8')

# Define a function to generate a customer
def generate_customer_profile():
    profile = fake.profile()
    return {
        "name": profile['name'],
        "address": fake.address().replace('\n', ', '),
        "email": profile['mail'],
        "phone_number": fake.phone_number(),
        "social_security_number": fake.ssn(),
        "job": profile['job'],
        "company": profile['company'],
        "residence": fake.address().replace('\n', ', '),
        "current_location": fake.address().replace('\n', ', '),
        "website": profile['website'][0] if profile['website'] else None,
        "username": profile['username'],
        "age": fake.random_int(min=18, max=80, step=1),
        "gender" : profile['sex'],
        "Grades": fake.random_int(min=0, max=100, step=1),
        "preferred_language": fake.language_code(),
        "preferred_currency": fake.currency_code(),
        "job_title": profile['job'],
        "national_id": fake.random_int(min=1000000000, max=9999999999, step=1),
        "height": fake.random_int(min=100, max=200, step=1),
        "weight": fake.random_int(min=100, max=200, step=1),
    }

# Define a function to generate a transaction
def generate_transaction(name):
    return {
        "name": name,
        "amount": f"${fake.random_int(min=1, max=1000, step=1):.2f}",
        "date": fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None),
    }

# Generate customers
customers = [generate_customer_profile() for _ in range(10)]
customers_df = pd.DataFrame(customers)



In [70]:

# Generate transactions for each customer randomly picking a customer by name
transactions = [generate_transaction(random.choice(customers)["name"]) for _ in range(20)]
transactions_df = pd.DataFrame(transactions)
transactions_df.head()

,name,amount,date
0,Mrs. Danielle Gonzalez,$535.00,2023-02-11 21:14:40
1,Mrs. Danielle Gonzalez,$828.00,2022-12-01 14:52:27
2,Patricia Diaz,$258.00,2023-03-22 16:34:19
3,Vanessa Lewis,$133.00,2023-04-04 23:54:43
4,Alexander Watkins,$97.00,2022-08-19 19:32:15


In [71]:

# Encrypt the data in the CRM
password = get_random_bytes(16)  # In a real system, never hard-code the password
cipher = AESCipher(password)


In [72]:

encrypted_df = pd.DataFrame()
encryption_key = pd.DataFrame()
for column in customers_df.columns:
    encrypted_column = []
    key_column = []
    for data in customers_df[column]:
        iv, ct = cipher.encrypt(str(data).encode('utf-8'))
        encrypted_column.append(ct)
        key_column.append(iv)
    encrypted_df[column] = encrypted_column
    encryption_key[column] = key_column

print("password: ", password)
# Print the encrypted data
encrypted_df.head()

password:  b'\x7f>\x7f*\x98f\x9dC^\xc6\x03\xa4\xb3,}\xc8'


,name,address,email,phone_number,social_security_number,job,company,residence,current_location,website,username,age,gender,Grades,preferred_language,preferred_currency,job_title,national_id,height,weight
0,hsMT52akCnF+K4mXOA==,VrGUOPYRYVl7k+X0oD0ChraVNgt4ghXXucalTiMEEOKs72...,/X9nc2/286H62iZZ7grQxipU6w==,8i8xmxvpgVo6sm04,Riy0GNBnFJpCnoo=,ey8EPyHSfZ73JMmtStbPm0lT5R6fj2/chvrB,2S0o9uD0iPk1Hg==,vWkelgwbV+cjjJPWYd5t+SxHIzlw11VL8tKhGPs+LiPQZb...,fscu1jPCqAPnAAcNrbzQolOX8tzAowJajnHiP3Turb0r8A...,CY7+iLtNwUkJuG75yVQsolteDqN/KOqOAA==,dKitM4+gsSlj,M00=,zQ==,NPg=,QbI=,Z0ON,6yv8bb+Dtvj3uWKbjENDGXqG3MR42yn516JS,ecxcCn1Ih7aFYA==,J4I5,2+Ql
1,04JJnyd9Cgxu+a3NMFrUsfc=,Wkq8DNIeU500sG4RFjCtuxRFDE1stseej3cxRBxxXecygk...,4yoIZNmqYkVZ+Yu4Q2G6ZSM=,mHpdR/ilVjmhZQ==,EVzjbId0omn2r5A=,7cojfKPqTA0LE6CEleP5rsja47GLOQ==,Ri/waex00UjXPFoC0X6fChU=,/SGzyCio9l7N9EcIVS02tEcfiXLgPzqD+MvBHFVutycZiV...,iX7QOp9kJvxbwC/JwL0RJbLsqLK/Ruq5zGwjBE5Jf8R1re...,CI8GMhNhSyZYPqphbTWL4ij1QNejEA==,uQFdBilWw2E=,v68=,Vg==,yKE=,LO1s,cdKb,2/6rOAtDmnh6jFWUiNPvyME/jvx9XQ==,u+lPX2Q1eGgIiw==,2N6u,eifv
2,ahLv9BuBm63Sw/c=,rjOMMJDO1DfFf3X3cp/wwtOXSqGQ0wWAg6N+xB2Dts1Xi9...,hAu7tMYMxwShii0ikkUKrZIa,eyM2rWP/RoUg9Gcfjz4shu6z,QjQBveJj2cNmJcY=,+rYPepfdLOffQvcc3kTGffrMV54GUxVV0t0ec9FThbg=,dd1SCQJCqVg1oK3t,TAWZ9LI0oyRWl21k5pPvrJiTgQnnA1y/8Tucc/WgX9sy40...,q4V2/7oDCunKjWvYoHVQLf4fMUWauHe1k6i/e9dLby2a40...,k6dXOUMKqW6WZ9Bwtxq5uu8Wahzlqc8F3UgoljzO,JF4KKRxodDEvdZ8=,B80=,cw==,TOg=,yGc=,47gr,sGD1UPFR9SJzJX4ua0OZD2UGUdYXrmdTEDgWB+Ef60g=,T5jGK2wE3fNg8w==,73nC,wgMk
3,wpooEuPGBAu/tMfN5QU=,Of0zPRjriU9QZ6qB9TdThuo811GCScHfuaYpoVan9r2yw8...,Y1ocqaYGk5BMbrHR/X7m5Pyy,0JKctMYKcJuO3AcheuU3l+w24A==,MdXBXFHBTBfO8Kk=,T89nLJPy8sCzaeqbXCvZmdphd/I=,Jxb9qeIO7aq18LUQtV8=,JE0KNqzWWKcUq5SX0QMKJ4S4jAhrl7h2RRvaPAdKjDDscp...,AhJ6EJuAb0mhPgQMu5Y/xhoWksIwzBTRU1hT4HABxLRl1pA=,H8vIYS+ICqapmpI/NDWL6UrI0kCdLdnqnXq8,vJafC+QT8eKMqLnkpA==,4A4=,pA==,dpo=,qZw=,PNw+,3ToeLQNPgGzI1FwZ/EK+Eg+uZko=,ZRNsDeNZuxkhGw==,9vrX,d56f
4,jV+lr3GIHFyiNbbU,dgTVp8nX9i6OW+k54rlQkKbqycid2y/cwb0b9XmbC1G0zI...,ofGwstc+wNmEGvmp0G1OzOVjgJI=,jXArbz/zWC8GU06aZVWHjfxt,23RiWd4r1/trM2Q=,PwpC8O/cQQAtrrEIVA==,5UxEW0P+sDCiXOKL8ntP,1Pcr+f7XNKv8KIXjHS4NNHHTdCDKog==,/ClFow7b28HIUctQEjmvezsg/KzamIFitoOI+ZsUvxNizC...,BO4N8e31rFM7syUvU/aKwaAWdk3o,5DZJDYQZUaP6,QCE=,7w==,lts=,EpY=,cA+y,PXYF+v+Wtj4k4+9qhw==,iG/ZbDCNAUWzdg==,fUpk,yvFt
